In [1]:
from src.data_application import DataFrameProcessor
from src.data_local import DataLoader, MergedDataPreprocessing
from cashe import get_input

In [2]:
df_original = get_input()

In [3]:
df_original.head(5)

,item_CreatedDate,transaction_RequestId,transaction_PatientAge,transaction_PatientEnGender,item_NameEn,item_Price,item_Status,item_Sequence,item_RequestQuantity,item_ResponseQuantity,transaction_DiagnosisIds,transaction_PhysicianIds,item_ResponseState
0,2023-08-24 14:21:05.369,75a9840c-865f-4fd9-9b67-255bb8771eee,6Y-3M,Male,Examination,80.0,1,1,1.0,0.0,J02,",327",NaN
1,2023-08-24 14:34:14.349,6c46f638-517f-493d-b710-4aef5f9e9393,6Y-3M,Male,Examination,80.0,1,1,1.0,0.0,J02,",327",NaN
2,2023-08-24 14:58:18.912,3c2813fe-f1f3-438f-b16f-92fcf917dc67,6Y-3M,Male,Examination,80.0,1,1,1.0,0.0,J02,",327",NaN
3,2023-08-24 15:00:39.525,65e81315-23b8-4d4e-9767-040c45a0f560,6Y-3M,Male,Examination,80.0,1,1,1.0,0.0,J02,",327",NaN
4,2023-08-24 15:03:23.404,c4427a02-3291-422d-b2e8-b97bf3d5081d,6Y-3M,Male,Examination,80.0,2,1,1.0,1.0,J02,",327",approved


In [4]:
preprocessing = MergedDataPreprocessing(df_original)

In [5]:
df_original_prep = preprocessing.age_gender_item_ids_prep()
df_original_prep

E:\Projects\Claims_Rejection\src\data_local.py:164: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  self.df.item_Price.fillna(column_average, inplace=True)


,item_CreatedDate,transaction_RequestId,transaction_PatientAge,transaction_PatientEnGender,item_NameEn,item_Price,item_Status,item_Sequence,item_RequestQuantity,item_ResponseQuantity,transaction_DiagnosisIds,transaction_PhysicianIds,item_ResponseState
0,2023-08-24 14:21:05.369,75a9840c-865f-4fd9-9b67-255bb8771eee,6,1,74,80.00,1,1,1.0,0.0,7,",327",NaN
1,2023-08-24 14:34:14.349,6c46f638-517f-493d-b710-4aef5f9e9393,6,1,74,80.00,1,1,1.0,0.0,7,",327",NaN
2,2023-08-24 14:58:18.912,3c2813fe-f1f3-438f-b16f-92fcf917dc67,6,1,74,80.00,1,1,1.0,0.0,7,",327",NaN
3,2023-08-24 15:00:39.525,65e81315-23b8-4d4e-9767-040c45a0f560,6,1,74,80.00,1,1,1.0,0.0,7,",327",NaN
4,2023-08-24 15:03:23.404,c4427a02-3291-422d-b2e8-b97bf3d5081d,6,1,74,80.00,2,1,1.0,1.0,7,",327",approved
...,...,...,...,...,...,...,...,...,...,...,...,...,...
49087,2024-04-30 09:00:16.704,f4cbab83-28f3-4410-948c-2c87cdcdd8da,31,1,222,121.00,1,3,1.0,0.0,2,",238,238,238,238,238,238,238,238,238,238,238,2...",NaN
49088,2024-04-30 09:00:16.704,f4cbab83-28f3-4410-948c-2c87cdcdd8da,31,1,234,60.50,1,2,1.0,0.0,2,",238,238,238,238,238,238,238,238,238,238,238,2...",NaN
49089,2024-04-30 09:00:16.704,f4cbab83-28f3-4410-948c-2c87cdcdd8da,31,1,74,80.00,1,1,1.0,0.0,2,",238,238,238,238,238,238,238,238,238,238,238,2...",NaN
49090,2024-04-30 09:00:16.704,f4cbab83-28f3-4410-948c-2c87cdcdd8da,31,1,41,54.25,1,16,1.0,0.0,2,",238,238,238,238,238,238,238,238,238,238,238,2...",NaN


In [6]:
df_train, df_test = preprocessing.train_test_split(startify_column='item_CreatedDate')

In [7]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14976 entries, 8355 to 38075
Data columns (total 13 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   item_CreatedDate             14976 non-null  object 
 1   transaction_RequestId        14976 non-null  object 
 2   transaction_PatientAge       14976 non-null  int64  
 3   transaction_PatientEnGender  14976 non-null  int64  
 4   item_NameEn                  14976 non-null  int32  
 5   item_Price                   14976 non-null  float64
 6   item_Status                  14976 non-null  int64  
 7   item_Sequence                14976 non-null  int64  
 8   item_RequestQuantity         14976 non-null  float64
 9   item_ResponseQuantity        14976 non-null  float64
 10  transaction_DiagnosisIds     14976 non-null  int32  
 11  transaction_PhysicianIds     14976 non-null  object 
 12  item_ResponseState           14976 non-null  object 
dtypes: float64(3), int

In [8]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3745 entries, 17344 to 24635
Data columns (total 13 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   item_CreatedDate             3745 non-null   object 
 1   transaction_RequestId        3745 non-null   object 
 2   transaction_PatientAge       3745 non-null   int64  
 3   transaction_PatientEnGender  3745 non-null   int64  
 4   item_NameEn                  3745 non-null   int32  
 5   item_Price                   3745 non-null   float64
 6   item_Status                  3745 non-null   int64  
 7   item_Sequence                3745 non-null   int64  
 8   item_RequestQuantity         3745 non-null   float64
 9   item_ResponseQuantity        3745 non-null   float64
 10  transaction_DiagnosisIds     3745 non-null   int32  
 11  transaction_PhysicianIds     3745 non-null   object 
 12  item_ResponseState           3745 non-null   object 
dtypes: float64(3), int

In [9]:
df_train.to_excel('data/SplittedData/SNB_train_data.xlsx',index=False)
df_test.to_excel('data/SplittedData/SNB_test_data.xlsx',index=False)

In [ ]:
#df_trans_req, df_trans_res = data_loader.split_transaction(df_trans)

In [ ]:
df_trans_req = DataFrameProcessor(df_trans_req).filter_high_variance_features()
df_trans_res = DataFrameProcessor(df_trans_res).filter_high_variance_features()

In [ ]:
df_trans_req.head(3)

In [ ]:
df_trans_req.columns

In [ ]:
df_trans_res.head(3)

In [ ]:
data_loader = DataLoader()
df_request, df_response = data_loader.load_data()

In [ ]:
len(df_request)

In [ ]:
processor = DataFrameProcessor(df_request)

df_request_out = processor.filter_high_variance_features()

In [ ]:
df_request.to_excel('sad.xlsx')

In [ ]:
print(1)

In [ ]:
df_request_out.head(3)

In [ ]:
df_request_out.columns

In [ ]:
dfs[1].head()

In [ ]:
dfs[2].head()